####  1. Select Stock Ticker

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Set the ticker symbol
ticker = "INFY.NS"

#### 2. Select appropriate device

In [ ]:
import torch

# Select appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#### 3. Run the CUDA check

In [ ]:
from Config.check_cuda_config import check_cuda_configuration

# Run the CUDA check
check_cuda_configuration()

#### 4. Fetch stock data

In [ ]:
import os

from DataPipeline.data_fetcher import fetch_daily_data_ist

# Create the path if it doesn't exist
raw_data_dir = "Data/RawData"

# Fetch stock data
data = fetch_daily_data_ist(ticker)

# Construct filename and full path
filename = f"{ticker[:-3]}.csv"
filepath = os.path.join(raw_data_dir, filename)

# Save to CSV
data.to_csv(filepath)
print(f"Fetched raw data saved to: {filepath}")
data.head()

#### 5. Calculate Technical Indicators

In [ ]:
from DataPipeline.technical_indicators import TechnicalIndicators

# Calculate technical indicators
indicators = TechnicalIndicators(data)
indicators_data = indicators.calculate_all()

#### 6. Data Splitting

In [ ]:
print(f"\nProcessed data shape after indicators: {indicators_data.shape}")

# Split data into train and validation (85%) and test (15%)
total_size = len(indicators_data)
train_size = int(0.85 * total_size)

train_data = indicators_data[:train_size].copy()
test_data = indicators_data[train_size:].copy()

print(f"\nData split sizes:")
print(f"\nTrain: {len(train_data)}, Test: {len(test_data)}")

#### 7. Dropping NaN Columns and Rows

In [ ]:
from DataPipeline.data_cleaning import drop_all_null_columns

# Drop all-null columns
train_data_ready, dropped_columns = drop_all_null_columns(train_data)
test_data_ready, _ = drop_all_null_columns(test_data)

print(f"\nDropped columns due to nulls: {dropped_columns}")
print(f"Final shapes - Train: {train_data_ready.shape}, Test: {test_data_ready.shape}")

#### 8. Data Preparation

In [ ]:
from Optimization.prepare_data_for_hpo import prepare_data_for_hpo

# Prepare data for LSTM (only train loader, no val)
data, train_loader, val_loader, X_scaler, y_scaler, feature_names, num_features = prepare_data_for_hpo(
    Modelling_data=train_data_ready,
    model_type="LSTM",
    batch_size=64,
    seq_length=30
)

#### 9. Hyper Parameter Optimization

In [ ]:
from Optimization.hyperparameter_optimization import run_hyperparameter_optimization
data_file = train_data_ready  # Replace with your actual data file
model_type = "lstm"
n_trials = 2
output_dir = "Results/HPO"

# Print welcome message
print(f"Starting Bayesian HPO for {model_type.upper()} model with {n_trials} trials")
print(f"Data file: {data_file}")
print(f"Output directory: {output_dir}")

# Run hyperparameter optimization
best_params, study = run_hyperparameter_optimization(
    data_file, 
    output_dir=output_dir, 
    n_trials=n_trials, 
    model_type=model_type
)

print("\nOptimization complete!")
print(f"Best validation RMSE: {study.best_value:.8f}")
print(f"Best parameters: {best_params}")



#### 10. Test Set Predictions

In [ ]:
# ====================================
# Imports
# ====================================
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Custom modules
from Models.evaluate_model import evaluate_model
from Optimization.select_features_for_model import select_features_for_model
from PlotScripts.get_time_series_comparison_plot import time_series_comparison_plot
from PlotScripts.get_scatter_plot import scatter_plot
from PlotScripts.get_residual_analysis_plot import residual_analysis


# ====================================
# Step 1: Data Preparation
# ====================================
df = test_data_ready.copy()  # Work on a copy to preserve original
valid_features = select_features_for_model(df, "lstm")  # Select features suitable for LSTM
seq_length = 30  # Length of input sequences


# ====================================
# Step 2: Feature & Target Scaling
# ====================================
X_all = X_scaler.transform(df[valid_features].values)   # Feature scaling
y_all = df["Close"].values.reshape(-1, 1)               # Target variable
y_all_scaled = y_scaler.transform(y_all)                # Target scaling


# ====================================
# Step 3: Create Sliding Windows (Sequence Generation)
# ====================================
X_windows = []  # Sequences of input features
y_targets = []  # Corresponding next-day targets

for end_ix in range(seq_length, len(X_all)):
    start_ix = end_ix - seq_length
    X_windows.append(X_all[start_ix:end_ix])
    y_targets.append(y_all_scaled[end_ix])

X_tensor = torch.tensor(X_windows, dtype=torch.float32)
y_tensor = torch.tensor(y_targets, dtype=torch.float32)


# ====================================
# Step 4: DataLoader Creation
# ====================================
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)


# ====================================
# Step 5: Model Evaluation
# ====================================
metrics = evaluate_model(Trained_Model, data_loader=loader)

# ====================================
# Step 6: Model Inference for Plotting
# ====================================
Trained_Model.eval()  # Set model to evaluation mode
X_tensor = X_tensor.to(device)

with torch.no_grad():
    y_pred_scaled = Trained_Model(X_tensor).view(-1, 1).cpu().numpy()

# Inverse scale to get actual price values
y_pred = y_scaler.inverse_transform(y_pred_scaled).flatten()
y_true = y_all[seq_length:].flatten()
dates = df.index[seq_length:]


# ====================================
# Step 7: Visualization
# ====================================
time_series_comparison_plot(
    targets_original=y_true,
    predictions_original=y_pred,
    model_type="LSTM",
    output_dir="Results",
    phase="Testing"
)
scatter_plot(
    targets_original=y_true,
    predictions_original=y_pred,
    model_type="LSTM",
    output_dir="Results",
    phase="Testing"
)
residual_analysis(
    targets_original=y_true,
    predictions_original=y_pred,
    model_type="LSTM",
    output_dir="Results",
    phase="Testing"
)

# ====================================
# Step 8: Store and Print Results
# ====================================
results = {
    f"{ticker}_predictions": y_pred,
    f"{ticker}_actuals": y_true,
    **{f"{ticker}_{k}": v for k, v in metrics.items()},
}


####  11. Generate Forecast 

In [ ]:
# Import the forecasting function from the utility module
from Utils.get_forecast import generate_forecast

# Create a copy of the test data to avoid modifying the original dataset
forecast_data = test_data_ready.copy()

# Generate forecast using the trained LSTM model
generate_forecast(
    forecast_data=forecast_data,  # Input data prepared for forecasting
    Trained_Model=Trained_Model,  # The trained model used for prediction
    X_scaler=X_scaler,  # Scaler used to normalize input features
    y_scaler=y_scaler,  # Scaler used to denormalize output predictions
    seq_length=30,  # Number of time steps to look back for each prediction
    ticker=ticker,  # Ticker symbol for the stock (or identifier for the time series)
    model="lstm",  # Type of model to be used ("lstm" in this case)
)